In [ ]:
%pip install "pydantic>=2.7.1,<3.0.0" "python-multipart==0.0.9" "python-socketio>=5.12.1,<6.0.0" "grpcio>=1.71.0,<2.0.0" "passlib>=1.7.4,<2.0.0" "pyjwt>=2.10.1,<3.0.0" "requests==2.32.2" "aiohttp==3.9.5" "httpx>=0.28.1,<1.0.0" "openai>=1.66.3,<2.0.0" "langgraph>=0.3.18,<1.0.0" "langchain-core>=0.3.47,<1.0.0" "langchain-openai>=0.3.9,<1.0.0" "langchain>=0.3.23,<1.0.0" "pandas>=2.2.3,<3.0.0" "psycopg2>=2.9.10,<3.0.0" "sqlalchemy>=2.0.40,<3.0.0" "scipy>=1.15.2,<2.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.8/151.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.

In [ ]:
%pip install dotenv
%pip install gradio

  Attempting uninstall: python-multipart
    Found existing installation: python-multipart 0.0.9
    Uninstalling python-multipart-0.0.9:
      Successfully uninstalled python-multipart-0.0.9


In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY2')

# Tools

In [ ]:
#!/usr/bin/env python
# coding=utf-8

import ast
import builtins
import difflib
import inspect
import logging
import math
import re
from collections.abc import Mapping
from functools import wraps
from importlib import import_module
from types import BuiltinFunctionType, FunctionType, ModuleType
from typing import Any, Callable, Dict, List, Optional, Set, Tuple

logger = logging.getLogger(__name__)


BASE_BUILTIN_MODULES = [
    "collections",
    "datetime",
    "itertools",
    "math",
    "queue",
    "random",
    "re",
    "stat",
    "statistics",
    "time",
    "unicodedata",
    "sympy",
    "pandas",
    "matplotlib.pyplot","seaborn",
]

def custom_print(*args):
    return None

BASE_PYTHON_TOOLS = {
    "print": custom_print,
    "isinstance": isinstance,
    "range": range,
    "float": float,
    "int": int,
    "bool": bool,
    "str": str,
    "set": set,
    "list": list,
    "dict": dict,
    "tuple": tuple,
    "round": round,
    "ceil": math.ceil,
    "floor": math.floor,
    "log": math.log,
    "exp": math.exp,
    "sin": math.sin,
    "cos": math.cos,
    "tan": math.tan,
    "asin": math.asin,
    "acos": math.acos,
    "atan": math.atan,
    "atan2": math.atan2,
    "degrees": math.degrees,
    "radians": math.radians,
    "pow": pow,
    "sqrt": math.sqrt,
    "len": len,
    "sum": sum,
    "max": max,
    "min": min,
    "abs": abs,
    "enumerate": enumerate,
    "zip": zip,
    "reversed": reversed,
    "sorted": sorted,
    "all": all,
    "any": any,
    "map": map,
    "filter": filter,
    "ord": ord,
    "chr": chr,
    "next": next,
    "iter": iter,
    "divmod": divmod,
    "callable": callable,
    "getattr": getattr,
    "hasattr": hasattr,
    "setattr": setattr,
    "issubclass": issubclass,
    "type": type,
    "complex": complex,
}

MAX_LENGTH_TRUNCATE_CONTENT = 20000

def truncate_content(content: str, max_length: int = MAX_LENGTH_TRUNCATE_CONTENT) -> str:
    if len(content) <= max_length:
        return content
    else:
        return (
            content[: max_length // 2]
            + f"\n..._This content has been truncated to stay below {max_length} characters_...\n"
            + content[-max_length // 2 :]
        )


class InterpreterError(ValueError):
    """
    An error raised when the interpreter cannot evaluate a Python expression, due to syntax error or unsupported
    operations.
    """

    pass


ERRORS = {
    name: getattr(builtins, name)
    for name in dir(builtins)
    if isinstance(getattr(builtins, name), type) and issubclass(getattr(builtins, name), BaseException)
}

DEFAULT_MAX_LEN_OUTPUT = 50000
MAX_OPERATIONS = 10000000
MAX_WHILE_ITERATIONS = 1000000


def custom_print(*args):
    return None


BASE_PYTHON_TOOLS = {
    "print": custom_print,
    "isinstance": isinstance,
    "range": range,
    "float": float,
    "int": int,
    "bool": bool,
    "str": str,
    "set": set,
    "list": list,
    "dict": dict,
    "tuple": tuple,
    "round": round,
    "ceil": math.ceil,
    "floor": math.floor,
    "log": math.log,
    "exp": math.exp,
    "sin": math.sin,
    "cos": math.cos,
    "tan": math.tan,
    "asin": math.asin,
    "acos": math.acos,
    "atan": math.atan,
    "atan2": math.atan2,
    "degrees": math.degrees,
    "radians": math.radians,
    "pow": pow,
    "sqrt": math.sqrt,
    "len": len,
    "sum": sum,
    "max": max,
    "min": min,
    "abs": abs,
    "enumerate": enumerate,
    "zip": zip,
    "reversed": reversed,
    "sorted": sorted,
    "all": all,
    "any": any,
    "map": map,
    "filter": filter,
    "ord": ord,
    "chr": chr,
    "next": next,
    "iter": iter,
    "divmod": divmod,
    "callable": callable,
    "getattr": getattr,
    "hasattr": hasattr,
    "setattr": setattr,
    "issubclass": issubclass,
    "type": type,
    "complex": complex,
}

DANGEROUS_FUNCTIONS = [
    "builtins.compile",
    "builtins.eval",
    "builtins.exec",
    "builtins.globals",
    "builtins.locals",
    "builtins.__import__",
    "os.popen",
    "os.system",
    "posix.system",
]


class PrintContainer:
    def __init__(self):
        self.value = ""

    def append(self, text):
        self.value += text
        return self

    def __iadd__(self, other):
        """Implements the += operator"""
        self.value += str(other)
        return self

    def __str__(self):
        """String representation"""
        return self.value

    def __repr__(self):
        """Representation for debugging"""
        return f"PrintContainer({self.value})"

    def __len__(self):
        """Implements len() function support"""
        return len(self.value)


class BreakException(Exception):
    pass


class ContinueException(Exception):
    pass


class ReturnException(Exception):
    def __init__(self, value):
        self.value = value


def get_iterable(obj):
    if isinstance(obj, list):
        return obj
    elif hasattr(obj, "__iter__"):
        return list(obj)
    else:
        raise InterpreterError("Object is not iterable")


def fix_final_answer_code(code: str) -> str:
    """
    Sometimes an LLM can try to assign a variable to final_answer, which would break the final_answer() tool.
    This function fixes this behaviour by replacing variable assignments to final_answer with final_answer_variable,
    while preserving function calls to final_answer().
    """
    # First, find if there's a direct assignment to final_answer
    # Use word boundary and negative lookbehind to ensure it's not an object attribute
    assignment_pattern = r"(?<!\.)(?<!\w)\bfinal_answer\s*="
    if "final_answer(" not in code or not re.search(assignment_pattern, code):
        # If final_answer tool is not called in this blob, then doing the replacement is hazardous because it could false the model's memory for next steps.
        # Let's not modify the code and leave the subsequent assignment error happen.
        return code

    # Pattern for replacing variable assignments
    # Looks for 'final_answer' followed by '=' with optional whitespace
    # Negative lookbehind ensures we don't match object attributes
    assignment_regex = r"(?<!\.)(?<!\w)(\bfinal_answer)(\s*=)"
    code = re.sub(assignment_regex, r"final_answer_variable\2", code)

    # Pattern for replacing variable usage but not function calls
    # Negative lookahead (?!\s*\() ensures we don't match function calls
    # Negative lookbehind (?<!\.|\w) ensures we don't match object methods or other variables
    variable_regex = r"(?<!\.)(?<!\w)(\bfinal_answer\b)(?!\s*\()"
    code = re.sub(variable_regex, "final_answer_variable", code)
    return code


def safer_eval(func: Callable):
    """
    Decorator to make the evaluation of a function safer by checking its return value.

    Args:
        func: Function to make safer.

    Returns:
        Callable: Safer function with return value check.
    """

    @wraps(func)
    def _check_return(
        expression,
        state,
        static_tools,
        custom_tools,
        authorized_imports=BASE_BUILTIN_MODULES,
    ):
        result = func(expression, state, static_tools, custom_tools, authorized_imports=authorized_imports)
        if "*" not in authorized_imports:
            if isinstance(result, ModuleType):
                if result.__name__ not in authorized_imports:
                    raise InterpreterError(f"Forbidden access to module: {result.__name__}")
            elif isinstance(result, dict) and result.get("__spec__"):
                if result["__name__"] not in authorized_imports:
                    raise InterpreterError(f"Forbidden access to module: {result['__name__']}")
            elif isinstance(result, (FunctionType, BuiltinFunctionType)):
                for qualified_function_name in DANGEROUS_FUNCTIONS:
                    module_name, function_name = qualified_function_name.rsplit(".", 1)
                    if (
                        function_name not in static_tools
                        and result.__name__ == function_name
                        and result.__module__ == module_name
                    ):
                        raise InterpreterError(f"Forbidden access to function: {function_name}")
        return result

    return _check_return


def evaluate_attribute(
    expression: ast.Attribute,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> Any:
    if expression.attr.startswith("__") and expression.attr.endswith("__"):
        raise InterpreterError(f"Forbidden access to dunder attribute: {expression.attr}")
    value = evaluate_ast(expression.value, state, static_tools, custom_tools, authorized_imports)
    return getattr(value, expression.attr)


def evaluate_unaryop(
    expression: ast.UnaryOp,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> Any:
    operand = evaluate_ast(expression.operand, state, static_tools, custom_tools, authorized_imports)
    if isinstance(expression.op, ast.USub):
        return -operand
    elif isinstance(expression.op, ast.UAdd):
        return operand
    elif isinstance(expression.op, ast.Not):
        return not operand
    elif isinstance(expression.op, ast.Invert):
        return ~operand
    else:
        raise InterpreterError(f"Unary operation {expression.op.__class__.__name__} is not supported.")


def evaluate_lambda(
    lambda_expression: ast.Lambda,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> Callable:
    args = [arg.arg for arg in lambda_expression.args.args]

    def lambda_func(*values: Any) -> Any:
        new_state = state.copy()
        for arg, value in zip(args, values):
            new_state[arg] = value
        return evaluate_ast(
            lambda_expression.body,
            new_state,
            static_tools,
            custom_tools,
            authorized_imports,
        )

    return lambda_func


def evaluate_while(
    while_loop: ast.While,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> None:
    iterations = 0
    while evaluate_ast(while_loop.test, state, static_tools, custom_tools, authorized_imports):
        for node in while_loop.body:
            try:
                evaluate_ast(node, state, static_tools, custom_tools, authorized_imports)
            except BreakException:
                return None
            except ContinueException:
                break
        iterations += 1
        if iterations > MAX_WHILE_ITERATIONS:
            raise InterpreterError(f"Maximum number of {MAX_WHILE_ITERATIONS} iterations in While loop exceeded")
    return None


def create_function(
    func_def: ast.FunctionDef,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> Callable:
    source_code = ast.unparse(func_def)

    def new_func(*args: Any, **kwargs: Any) -> Any:
        func_state = state.copy()
        arg_names = [arg.arg for arg in func_def.args.args]
        default_values = [
            evaluate_ast(d, state, static_tools, custom_tools, authorized_imports) for d in func_def.args.defaults
        ]

        # Apply default values
        defaults = dict(zip(arg_names[-len(default_values) :], default_values))

        # Set positional arguments
        for name, value in zip(arg_names, args):
            func_state[name] = value

        # Set keyword arguments
        for name, value in kwargs.items():
            func_state[name] = value

        # Handle variable arguments
        if func_def.args.vararg:
            vararg_name = func_def.args.vararg.arg
            func_state[vararg_name] = args

        if func_def.args.kwarg:
            kwarg_name = func_def.args.kwarg.arg
            func_state[kwarg_name] = kwargs

        # Set default values for arguments that were not provided
        for name, value in defaults.items():
            if name not in func_state:
                func_state[name] = value

        # Update function state with self and __class__
        if func_def.args.args and func_def.args.args[0].arg == "self":
            if args:
                func_state["self"] = args[0]
                func_state["__class__"] = args[0].__class__

        result = None
        try:
            for stmt in func_def.body:
                result = evaluate_ast(stmt, func_state, static_tools, custom_tools, authorized_imports)
        except ReturnException as e:
            result = e.value

        if func_def.name == "__init__":
            return None

        return result

    # Store original AST, source code, and name
    new_func.__ast__ = func_def
    new_func.__source__ = source_code
    new_func.__name__ = func_def.name

    return new_func


def evaluate_function_def(
    func_def: ast.FunctionDef,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> Callable:
    custom_tools[func_def.name] = create_function(func_def, state, static_tools, custom_tools, authorized_imports)
    return custom_tools[func_def.name]


def evaluate_class_def(
    class_def: ast.ClassDef,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> type:
    class_name = class_def.name
    bases = [evaluate_ast(base, state, static_tools, custom_tools, authorized_imports) for base in class_def.bases]
    class_dict = {}

    for stmt in class_def.body:
        if isinstance(stmt, ast.FunctionDef):
            class_dict[stmt.name] = evaluate_ast(stmt, state, static_tools, custom_tools, authorized_imports)
        elif isinstance(stmt, ast.Assign):
            for target in stmt.targets:
                if isinstance(target, ast.Name):
                    class_dict[target.id] = evaluate_ast(
                        stmt.value,
                        state,
                        static_tools,
                        custom_tools,
                        authorized_imports,
                    )
                elif isinstance(target, ast.Attribute):
                    class_dict[target.attr] = evaluate_ast(
                        stmt.value,
                        state,
                        static_tools,
                        custom_tools,
                        authorized_imports,
                    )
        else:
            raise InterpreterError(f"Unsupported statement in class body: {stmt.__class__.__name__}")

    new_class = type(class_name, tuple(bases), class_dict)
    state[class_name] = new_class
    return new_class


def evaluate_augassign(
    expression: ast.AugAssign,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> Any:
    def get_current_value(target: ast.AST) -> Any:
        if isinstance(target, ast.Name):
            return state.get(target.id, 0)
        elif isinstance(target, ast.Subscript):
            obj = evaluate_ast(target.value, state, static_tools, custom_tools, authorized_imports)
            key = evaluate_ast(target.slice, state, static_tools, custom_tools, authorized_imports)
            return obj[key]
        elif isinstance(target, ast.Attribute):
            obj = evaluate_ast(target.value, state, static_tools, custom_tools, authorized_imports)
            return getattr(obj, target.attr)
        elif isinstance(target, ast.Tuple):
            return tuple(get_current_value(elt) for elt in target.elts)
        elif isinstance(target, ast.List):
            return [get_current_value(elt) for elt in target.elts]
        else:
            raise InterpreterError("AugAssign not supported for {type(target)} targets.")

    current_value = get_current_value(expression.target)
    value_to_add = evaluate_ast(expression.value, state, static_tools, custom_tools, authorized_imports)

    if isinstance(expression.op, ast.Add):
        if isinstance(current_value, list):
            if not isinstance(value_to_add, list):
                raise InterpreterError(f"Cannot add non-list value {value_to_add} to a list.")
            current_value += value_to_add
        else:
            current_value += value_to_add
    elif isinstance(expression.op, ast.Sub):
        current_value -= value_to_add
    elif isinstance(expression.op, ast.Mult):
        current_value *= value_to_add
    elif isinstance(expression.op, ast.Div):
        current_value /= value_to_add
    elif isinstance(expression.op, ast.Mod):
        current_value %= value_to_add
    elif isinstance(expression.op, ast.Pow):
        current_value **= value_to_add
    elif isinstance(expression.op, ast.FloorDiv):
        current_value //= value_to_add
    elif isinstance(expression.op, ast.BitAnd):
        current_value &= value_to_add
    elif isinstance(expression.op, ast.BitOr):
        current_value |= value_to_add
    elif isinstance(expression.op, ast.BitXor):
        current_value ^= value_to_add
    elif isinstance(expression.op, ast.LShift):
        current_value <<= value_to_add
    elif isinstance(expression.op, ast.RShift):
        current_value >>= value_to_add
    else:
        raise InterpreterError(f"Operation {type(expression.op).__name__} is not supported.")

    # Update the state: current_value has been updated in-place
    set_value(
        expression.target,
        current_value,
        state,
        static_tools,
        custom_tools,
        authorized_imports,
    )

    return current_value


def evaluate_boolop(
    node: ast.BoolOp,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> bool:
    if isinstance(node.op, ast.And):
        for value in node.values:
            if not evaluate_ast(value, state, static_tools, custom_tools, authorized_imports):
                return False
        return True
    elif isinstance(node.op, ast.Or):
        for value in node.values:
            if evaluate_ast(value, state, static_tools, custom_tools, authorized_imports):
                return True
        return False


def evaluate_binop(
    binop: ast.BinOp,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> Any:
    # Recursively evaluate the left and right operands
    left_val = evaluate_ast(binop.left, state, static_tools, custom_tools, authorized_imports)
    right_val = evaluate_ast(binop.right, state, static_tools, custom_tools, authorized_imports)

    # Determine the operation based on the type of the operator in the BinOp
    if isinstance(binop.op, ast.Add):
        return left_val + right_val
    elif isinstance(binop.op, ast.Sub):
        return left_val - right_val
    elif isinstance(binop.op, ast.Mult):
        return left_val * right_val
    elif isinstance(binop.op, ast.Div):
        return left_val / right_val
    elif isinstance(binop.op, ast.Mod):
        return left_val % right_val
    elif isinstance(binop.op, ast.Pow):
        return left_val**right_val
    elif isinstance(binop.op, ast.FloorDiv):
        return left_val // right_val
    elif isinstance(binop.op, ast.BitAnd):
        return left_val & right_val
    elif isinstance(binop.op, ast.BitOr):
        return left_val | right_val
    elif isinstance(binop.op, ast.BitXor):
        return left_val ^ right_val
    elif isinstance(binop.op, ast.LShift):
        return left_val << right_val
    elif isinstance(binop.op, ast.RShift):
        return left_val >> right_val
    else:
        raise NotImplementedError(f"Binary operation {type(binop.op).__name__} is not implemented.")


def evaluate_assign(
    assign: ast.Assign,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> Any:
    result = evaluate_ast(assign.value, state, static_tools, custom_tools, authorized_imports)
    if len(assign.targets) == 1:
        target = assign.targets[0]
        set_value(target, result, state, static_tools, custom_tools, authorized_imports)
    else:
        expanded_values = []
        for tgt in assign.targets:
            if isinstance(tgt, ast.Starred):
                expanded_values.extend(result)
            else:
                expanded_values.append(result)

        for tgt, val in zip(assign.targets, expanded_values):
            set_value(tgt, val, state, static_tools, custom_tools, authorized_imports)
    return result


def set_value(
    target: ast.AST,
    value: Any,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> None:
    if isinstance(target, ast.Name):
        if target.id in static_tools:
            raise InterpreterError(f"Cannot assign to name '{target.id}': doing this would erase the existing tool!")
        state[target.id] = value
    elif isinstance(target, ast.Tuple):
        if not isinstance(value, tuple):
            if hasattr(value, "__iter__") and not isinstance(value, (str, bytes)):
                value = tuple(value)
            else:
                raise InterpreterError("Cannot unpack non-tuple value")
        if len(target.elts) != len(value):
            raise InterpreterError("Cannot unpack tuple of wrong size")
        for i, elem in enumerate(target.elts):
            set_value(elem, value[i], state, static_tools, custom_tools, authorized_imports)
    elif isinstance(target, ast.Subscript):
        obj = evaluate_ast(target.value, state, static_tools, custom_tools, authorized_imports)
        key = evaluate_ast(target.slice, state, static_tools, custom_tools, authorized_imports)
        obj[key] = value
    elif isinstance(target, ast.Attribute):
        obj = evaluate_ast(target.value, state, static_tools, custom_tools, authorized_imports)
        setattr(obj, target.attr, value)


def evaluate_call(
    call: ast.Call,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> Any:
    if not isinstance(call.func, (ast.Call, ast.Lambda, ast.Attribute, ast.Name, ast.Subscript)):
        raise InterpreterError(f"This is not a correct function: {call.func}).")

    func, func_name = None, None

    if isinstance(call.func, ast.Call):
        func = evaluate_ast(call.func, state, static_tools, custom_tools, authorized_imports)
    elif isinstance(call.func, ast.Lambda):
        func = evaluate_ast(call.func, state, static_tools, custom_tools, authorized_imports)
    elif isinstance(call.func, ast.Attribute):
        obj = evaluate_ast(call.func.value, state, static_tools, custom_tools, authorized_imports)
        func_name = call.func.attr
        if not hasattr(obj, func_name):
            raise InterpreterError(f"Object {obj} has no attribute {func_name}")
        func = getattr(obj, func_name)
    elif isinstance(call.func, ast.Name):
        func_name = call.func.id
        if func_name in state:
            func = state[func_name]
        elif func_name in static_tools:
            func = static_tools[func_name]
        elif func_name in custom_tools:
            func = custom_tools[func_name]
        elif func_name in ERRORS:
            func = ERRORS[func_name]
        else:
            raise InterpreterError(
                f"It is not permitted to evaluate other functions than the provided tools or functions defined/imported in previous code (tried to execute {call.func.id})."
            )
    elif isinstance(call.func, ast.Subscript):
        func = evaluate_ast(call.func, state, static_tools, custom_tools, authorized_imports)
        if not callable(func):
            raise InterpreterError(f"This is not a correct function: {call.func}).")
        func_name = None

    args = []
    for arg in call.args:
        if isinstance(arg, ast.Starred):
            args.extend(evaluate_ast(arg.value, state, static_tools, custom_tools, authorized_imports))
        else:
            args.append(evaluate_ast(arg, state, static_tools, custom_tools, authorized_imports))

    kwargs = {
        keyword.arg: evaluate_ast(keyword.value, state, static_tools, custom_tools, authorized_imports)
        for keyword in call.keywords
    }

    if func_name == "super":
        if not args:
            if "__class__" in state and "self" in state:
                return super(state["__class__"], state["self"])
            else:
                raise InterpreterError("super() needs at least one argument")
        cls = args[0]
        if not isinstance(cls, type):
            raise InterpreterError("super() argument 1 must be type")
        if len(args) == 1:
            return super(cls)
        elif len(args) == 2:
            instance = args[1]
            return super(cls, instance)
        else:
            raise InterpreterError("super() takes at most 2 arguments")
    elif func_name == "print":
        state["_print_outputs"] += " ".join(map(str, args)) + "\n"
        return None
    else:  # Assume it's a callable object
        if (inspect.getmodule(func) == builtins) and inspect.isbuiltin(func) and (func not in static_tools.values()):
            raise InterpreterError(
                f"Invoking a builtin function that has not been explicitly added as a tool is not allowed ({func_name})."
            )
        return func(*args, **kwargs)


def evaluate_subscript(
    subscript: ast.Subscript,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> Any:
    index = evaluate_ast(subscript.slice, state, static_tools, custom_tools, authorized_imports)
    value = evaluate_ast(subscript.value, state, static_tools, custom_tools, authorized_imports)
    try:
        return value[index]
    except (KeyError, IndexError, TypeError) as e:
        error_message = f"Could not index {value} with '{index}': {type(e).__name__}: {e}"
        if isinstance(index, str) and isinstance(value, Mapping):
            close_matches = difflib.get_close_matches(index, list(value.keys()))
            if len(close_matches) > 0:
                error_message += f". Maybe you meant one of these indexes instead: {str(close_matches)}"
        raise InterpreterError(error_message) from e


def evaluate_name(
    name: ast.Name,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> Any:
    if name.id in state:
        return state[name.id]
    elif name.id in static_tools:
        return static_tools[name.id]
    elif name.id in custom_tools:
        return custom_tools[name.id]
    elif name.id in ERRORS:
        return ERRORS[name.id]
    close_matches = difflib.get_close_matches(name.id, list(state.keys()))
    if len(close_matches) > 0:
        return state[close_matches[0]]
    raise InterpreterError(f"The variable `{name.id}` is not defined.")


def evaluate_condition(
    condition: ast.Compare,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> bool | object:
    result = True
    left = evaluate_ast(condition.left, state, static_tools, custom_tools, authorized_imports)
    for i, (op, comparator) in enumerate(zip(condition.ops, condition.comparators)):
        op = type(op)
        right = evaluate_ast(comparator, state, static_tools, custom_tools, authorized_imports)
        if op == ast.Eq:
            current_result = left == right
        elif op == ast.NotEq:
            current_result = left != right
        elif op == ast.Lt:
            current_result = left < right
        elif op == ast.LtE:
            current_result = left <= right
        elif op == ast.Gt:
            current_result = left > right
        elif op == ast.GtE:
            current_result = left >= right
        elif op == ast.Is:
            current_result = left is right
        elif op == ast.IsNot:
            current_result = left is not right
        elif op == ast.In:
            current_result = left in right
        elif op == ast.NotIn:
            current_result = left not in right
        else:
            raise InterpreterError(f"Unsupported comparison operator: {op}")

        if current_result is False:
            return False
        result = current_result if i == 0 else (result and current_result)
        left = right
    return result


def evaluate_if(
    if_statement: ast.If,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> Any:
    result = None
    test_result = evaluate_ast(if_statement.test, state, static_tools, custom_tools, authorized_imports)
    if test_result:
        for line in if_statement.body:
            line_result = evaluate_ast(line, state, static_tools, custom_tools, authorized_imports)
            if line_result is not None:
                result = line_result
    else:
        for line in if_statement.orelse:
            line_result = evaluate_ast(line, state, static_tools, custom_tools, authorized_imports)
            if line_result is not None:
                result = line_result
    return result


def evaluate_for(
    for_loop: ast.For,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> Any:
    result = None
    iterator = evaluate_ast(for_loop.iter, state, static_tools, custom_tools, authorized_imports)
    for counter in iterator:
        set_value(
            for_loop.target,
            counter,
            state,
            static_tools,
            custom_tools,
            authorized_imports,
        )
        for node in for_loop.body:
            try:
                line_result = evaluate_ast(node, state, static_tools, custom_tools, authorized_imports)
                if line_result is not None:
                    result = line_result
            except BreakException:
                break
            except ContinueException:
                continue
        else:
            continue
        break
    return result


def evaluate_listcomp(
    listcomp: ast.ListComp,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> List[Any]:
    def inner_evaluate(generators: List[ast.comprehension], index: int, current_state: Dict[str, Any]) -> List[Any]:
        if index >= len(generators):
            return [
                evaluate_ast(
                    listcomp.elt,
                    current_state,
                    static_tools,
                    custom_tools,
                    authorized_imports,
                )
            ]
        generator = generators[index]
        iter_value = evaluate_ast(
            generator.iter,
            current_state,
            static_tools,
            custom_tools,
            authorized_imports,
        )
        result = []
        for value in iter_value:
            new_state = current_state.copy()
            if isinstance(generator.target, ast.Tuple):
                for idx, elem in enumerate(generator.target.elts):
                    new_state[elem.id] = value[idx]
            else:
                new_state[generator.target.id] = value
            if all(
                evaluate_ast(if_clause, new_state, static_tools, custom_tools, authorized_imports)
                for if_clause in generator.ifs
            ):
                result.extend(inner_evaluate(generators, index + 1, new_state))
        return result

    return inner_evaluate(listcomp.generators, 0, state)


def evaluate_setcomp(
    setcomp: ast.SetComp,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> Set[Any]:
    result = set()
    for gen in setcomp.generators:
        iter_value = evaluate_ast(gen.iter, state, static_tools, custom_tools, authorized_imports)
        for value in iter_value:
            new_state = state.copy()
            set_value(
                gen.target,
                value,
                new_state,
                static_tools,
                custom_tools,
                authorized_imports,
            )
            if all(
                evaluate_ast(if_clause, new_state, static_tools, custom_tools, authorized_imports)
                for if_clause in gen.ifs
            ):
                element = evaluate_ast(
                    setcomp.elt,
                    new_state,
                    static_tools,
                    custom_tools,
                    authorized_imports,
                )
                result.add(element)
    return result


def evaluate_try(
    try_node: ast.Try,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> None:
    try:
        for stmt in try_node.body:
            evaluate_ast(stmt, state, static_tools, custom_tools, authorized_imports)
    except Exception as e:
        matched = False
        for handler in try_node.handlers:
            if handler.type is None or isinstance(
                e,
                evaluate_ast(handler.type, state, static_tools, custom_tools, authorized_imports),
            ):
                matched = True
                if handler.name:
                    state[handler.name] = e
                for stmt in handler.body:
                    evaluate_ast(stmt, state, static_tools, custom_tools, authorized_imports)
                break
        if not matched:
            raise e
    else:
        if try_node.orelse:
            for stmt in try_node.orelse:
                evaluate_ast(stmt, state, static_tools, custom_tools, authorized_imports)
    finally:
        if try_node.finalbody:
            for stmt in try_node.finalbody:
                evaluate_ast(stmt, state, static_tools, custom_tools, authorized_imports)


def evaluate_raise(
    raise_node: ast.Raise,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> None:
    if raise_node.exc is not None:
        exc = evaluate_ast(raise_node.exc, state, static_tools, custom_tools, authorized_imports)
    else:
        exc = None
    if raise_node.cause is not None:
        cause = evaluate_ast(raise_node.cause, state, static_tools, custom_tools, authorized_imports)
    else:
        cause = None
    if exc is not None:
        if cause is not None:
            raise exc from cause
        else:
            raise exc
    else:
        raise InterpreterError("Re-raise is not supported without an active exception")


def evaluate_assert(
    assert_node: ast.Assert,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> None:
    test_result = evaluate_ast(assert_node.test, state, static_tools, custom_tools, authorized_imports)
    if not test_result:
        if assert_node.msg:
            msg = evaluate_ast(assert_node.msg, state, static_tools, custom_tools, authorized_imports)
            raise AssertionError(msg)
        else:
            # Include the failing condition in the assertion message
            test_code = ast.unparse(assert_node.test)
            raise AssertionError(f"Assertion failed: {test_code}")


def evaluate_with(
    with_node: ast.With,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> None:
    contexts = []
    for item in with_node.items:
        context_expr = evaluate_ast(item.context_expr, state, static_tools, custom_tools, authorized_imports)
        if item.optional_vars:
            state[item.optional_vars.id] = context_expr.__enter__()
            contexts.append(state[item.optional_vars.id])
        else:
            context_var = context_expr.__enter__()
            contexts.append(context_var)

    try:
        for stmt in with_node.body:
            evaluate_ast(stmt, state, static_tools, custom_tools, authorized_imports)
    except Exception as e:
        for context in reversed(contexts):
            context.__exit__(type(e), e, e.__traceback__)
        raise
    else:
        for context in reversed(contexts):
            context.__exit__(None, None, None)


def get_safe_module(raw_module, authorized_imports, visited=None):
    """Creates a safe copy of a module or returns the original if it's a function"""
    # If it's a function or non-module object, return it directly
    if not isinstance(raw_module, ModuleType):
        return raw_module

    # Handle circular references: Initialize visited set for the first call
    if visited is None:
        visited = set()

    module_id = id(raw_module)
    if module_id in visited:
        return raw_module  # Return original for circular refs

    visited.add(module_id)

    # Create new module for actual modules
    safe_module = ModuleType(raw_module.__name__)

    # Copy all attributes by reference, recursively checking modules
    for attr_name in dir(raw_module):
        try:
            attr_value = getattr(raw_module, attr_name)
        except (ImportError, AttributeError) as e:
            # lazy / dynamic loading module -> INFO log and skip
            logger.info(
                f"Skipping import error while copying {raw_module.__name__}.{attr_name}: {type(e).__name__} - {e}"
            )
            continue
        # Recursively process nested modules, passing visited set
        if isinstance(attr_value, ModuleType):
            attr_value = get_safe_module(attr_value, authorized_imports, visited=visited)

        setattr(safe_module, attr_name, attr_value)

    return safe_module


def check_module_authorized(module_name, authorized_imports):
    if "*" in authorized_imports:
        return True
    else:
        module_path = module_name.split(".")
        # ["A", "B", "C"] -> ["A", "A.B", "A.B.C"]
        module_subpaths = [".".join(module_path[:i]) for i in range(1, len(module_path) + 1)]
        return any(subpath in authorized_imports for subpath in module_subpaths)


def evaluate_import(expression, state, authorized_imports):
    if isinstance(expression, ast.Import):
        for alias in expression.names:
            if check_module_authorized(alias.name, authorized_imports):
                raw_module = import_module(alias.name)
                state[alias.asname or alias.name] = get_safe_module(raw_module, authorized_imports)
            else:
                raise InterpreterError(
                    f"Import of {alias.name} is not allowed. Authorized imports are: {str(authorized_imports)}"
                )
        return None
    elif isinstance(expression, ast.ImportFrom):
        if check_module_authorized(expression.module, authorized_imports):
            raw_module = __import__(expression.module, fromlist=[alias.name for alias in expression.names])
            module = get_safe_module(raw_module, authorized_imports)
            if expression.names[0].name == "*":  # Handle "from module import *"
                if hasattr(module, "__all__"):  # If module has __all__, import only those names
                    for name in module.__all__:
                        state[name] = getattr(module, name)
                else:  # If no __all__, import all public names (those not starting with '_')
                    for name in dir(module):
                        if not name.startswith("_"):
                            state[name] = getattr(module, name)
            else:  # regular from imports
                for alias in expression.names:
                    if hasattr(module, alias.name):
                        state[alias.asname or alias.name] = getattr(module, alias.name)
                    else:
                        raise InterpreterError(f"Module {expression.module} has no attribute {alias.name}")
        else:
            raise InterpreterError(
                f"Import from {expression.module} is not allowed. Authorized imports are: {str(authorized_imports)}"
            )
        return None


def evaluate_dictcomp(
    dictcomp: ast.DictComp,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> Dict[Any, Any]:
    result = {}
    for gen in dictcomp.generators:
        iter_value = evaluate_ast(gen.iter, state, static_tools, custom_tools, authorized_imports)
        for value in iter_value:
            new_state = state.copy()
            set_value(
                gen.target,
                value,
                new_state,
                static_tools,
                custom_tools,
                authorized_imports,
            )
            if all(
                evaluate_ast(if_clause, new_state, static_tools, custom_tools, authorized_imports)
                for if_clause in gen.ifs
            ):
                key = evaluate_ast(
                    dictcomp.key,
                    new_state,
                    static_tools,
                    custom_tools,
                    authorized_imports,
                )
                val = evaluate_ast(
                    dictcomp.value,
                    new_state,
                    static_tools,
                    custom_tools,
                    authorized_imports,
                )
                result[key] = val
    return result


def evaluate_delete(
    delete_node: ast.Delete,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str],
) -> None:
    """
    Evaluate a delete statement (del x, del x[y]).

    Args:
        delete_node: The AST Delete node to evaluate
        state: The current state dictionary
        static_tools: Dictionary of static tools
        custom_tools: Dictionary of custom tools
        authorized_imports: List of authorized imports
    """
    for target in delete_node.targets:
        if isinstance(target, ast.Name):
            # Handle simple variable deletion (del x)
            if target.id in state:
                del state[target.id]
            else:
                raise InterpreterError(f"Cannot delete name '{target.id}': name is not defined")
        elif isinstance(target, ast.Subscript):
            # Handle index/key deletion (del x[y])
            obj = evaluate_ast(target.value, state, static_tools, custom_tools, authorized_imports)
            index = evaluate_ast(target.slice, state, static_tools, custom_tools, authorized_imports)
            try:
                del obj[index]
            except (TypeError, KeyError, IndexError) as e:
                raise InterpreterError(f"Cannot delete index/key: {str(e)}")
        else:
            raise InterpreterError(f"Deletion of {type(target).__name__} targets is not supported")


@safer_eval
def evaluate_ast(
    expression: ast.AST,
    state: Dict[str, Any],
    static_tools: Dict[str, Callable],
    custom_tools: Dict[str, Callable],
    authorized_imports: List[str] = BASE_BUILTIN_MODULES,
):
    """
    Evaluate an abstract syntax tree using the content of the variables stored in a state and only evaluating a given
    set of functions.

    This function will recurse through the nodes of the tree provided.

    Args:
        expression (`ast.AST`):
            The code to evaluate, as an abstract syntax tree.
        state (`Dict[str, Any]`):
            A dictionary mapping variable names to values. The `state` is updated if need be when the evaluation
            encounters assignments.
        static_tools (`Dict[str, Callable]`):
            Functions that may be called during the evaluation. Trying to change one of these static_tools will raise an error.
        custom_tools (`Dict[str, Callable]`):
            Functions that may be called during the evaluation. These static_tools can be overwritten.
        authorized_imports (`List[str]`):
            The list of modules that can be imported by the code. By default, only a few safe modules are allowed.
            If it contains "*", it will authorize any import. Use this at your own risk!
    """
    if state.setdefault("_operations_count", {"counter": 0})["counter"] >= MAX_OPERATIONS:
        raise InterpreterError(
            f"Reached the max number of operations of {MAX_OPERATIONS}. Maybe there is an infinite loop somewhere in the code, or you're just asking too many calculations."
        )
    state["_operations_count"]["counter"] += 1
    common_params = (state, static_tools, custom_tools, authorized_imports)
    if isinstance(expression, ast.Assign):
        # Assignment -> we evaluate the assignment which should update the state
        # We return the variable assigned as it may be used to determine the final result.
        return evaluate_assign(expression, *common_params)
    elif isinstance(expression, ast.AugAssign):
        return evaluate_augassign(expression, *common_params)
    elif isinstance(expression, ast.Call):
        # Function call -> we return the value of the function call
        return evaluate_call(expression, *common_params)
    elif isinstance(expression, ast.Constant):
        # Constant -> just return the value
        return expression.value
    elif isinstance(expression, ast.Tuple):
        return tuple((evaluate_ast(elt, *common_params) for elt in expression.elts))
    elif isinstance(expression, (ast.ListComp, ast.GeneratorExp)):
        return evaluate_listcomp(expression, *common_params)
    elif isinstance(expression, ast.DictComp):
        return evaluate_dictcomp(expression, *common_params)
    elif isinstance(expression, ast.SetComp):
        return evaluate_setcomp(expression, *common_params)
    elif isinstance(expression, ast.UnaryOp):
        return evaluate_unaryop(expression, *common_params)
    elif isinstance(expression, ast.Starred):
        return evaluate_ast(expression.value, *common_params)
    elif isinstance(expression, ast.BoolOp):
        # Boolean operation -> evaluate the operation
        return evaluate_boolop(expression, *common_params)
    elif isinstance(expression, ast.Break):
        raise BreakException()
    elif isinstance(expression, ast.Continue):
        raise ContinueException()
    elif isinstance(expression, ast.BinOp):
        # Binary operation -> execute operation
        return evaluate_binop(expression, *common_params)
    elif isinstance(expression, ast.Compare):
        # Comparison -> evaluate the comparison
        return evaluate_condition(expression, *common_params)
    elif isinstance(expression, ast.Lambda):
        return evaluate_lambda(expression, *common_params)
    elif isinstance(expression, ast.FunctionDef):
        return evaluate_function_def(expression, *common_params)
    elif isinstance(expression, ast.Dict):
        # Dict -> evaluate all keys and values
        keys = (evaluate_ast(k, *common_params) for k in expression.keys)
        values = (evaluate_ast(v, *common_params) for v in expression.values)
        return dict(zip(keys, values))
    elif isinstance(expression, ast.Expr):
        # Expression -> evaluate the content
        return evaluate_ast(expression.value, *common_params)
    elif isinstance(expression, ast.For):
        # For loop -> execute the loop
        return evaluate_for(expression, *common_params)
    elif isinstance(expression, ast.FormattedValue):
        # Formatted value (part of f-string) -> evaluate the content and format it
        value = evaluate_ast(expression.value, *common_params)
        # Early return if no format spec
        if not expression.format_spec:
            return value
        # Apply format specification
        format_spec = evaluate_ast(expression.format_spec, *common_params)
        return format(value, format_spec)
    elif isinstance(expression, ast.If):
        # If -> execute the right branch
        return evaluate_if(expression, *common_params)
    elif hasattr(ast, "Index") and isinstance(expression, ast.Index):
        return evaluate_ast(expression.value, *common_params)
    elif isinstance(expression, ast.JoinedStr):
        return "".join([str(evaluate_ast(v, *common_params)) for v in expression.values])
    elif isinstance(expression, ast.List):
        # List -> evaluate all elements
        return [evaluate_ast(elt, *common_params) for elt in expression.elts]
    elif isinstance(expression, ast.Name):
        # Name -> pick up the value in the state
        return evaluate_name(expression, *common_params)
    elif isinstance(expression, ast.Subscript):
        # Subscript -> return the value of the indexing
        return evaluate_subscript(expression, *common_params)
    elif isinstance(expression, ast.IfExp):
        test_val = evaluate_ast(expression.test, *common_params)
        if test_val:
            return evaluate_ast(expression.body, *common_params)
        else:
            return evaluate_ast(expression.orelse, *common_params)
    elif isinstance(expression, ast.Attribute):
        return evaluate_attribute(expression, *common_params)
    elif isinstance(expression, ast.Slice):
        return slice(
            evaluate_ast(expression.lower, *common_params) if expression.lower is not None else None,
            evaluate_ast(expression.upper, *common_params) if expression.upper is not None else None,
            evaluate_ast(expression.step, *common_params) if expression.step is not None else None,
        )
    elif isinstance(expression, ast.While):
        return evaluate_while(expression, *common_params)
    elif isinstance(expression, (ast.Import, ast.ImportFrom)):
        return evaluate_import(expression, state, authorized_imports)
    elif isinstance(expression, ast.ClassDef):
        return evaluate_class_def(expression, *common_params)
    elif isinstance(expression, ast.Try):
        return evaluate_try(expression, *common_params)
    elif isinstance(expression, ast.Raise):
        return evaluate_raise(expression, *common_params)
    elif isinstance(expression, ast.Assert):
        return evaluate_assert(expression, *common_params)
    elif isinstance(expression, ast.With):
        return evaluate_with(expression, *common_params)
    elif isinstance(expression, ast.Set):
        return set((evaluate_ast(elt, *common_params) for elt in expression.elts))
    elif isinstance(expression, ast.Return):
        raise ReturnException(evaluate_ast(expression.value, *common_params) if expression.value else None)
    elif isinstance(expression, ast.Pass):
        return None
    elif isinstance(expression, ast.Delete):
        return evaluate_delete(expression, *common_params)
    else:
        # For now we refuse anything else. Let's add things as we need them.
        raise InterpreterError(f"{expression.__class__.__name__} is not supported.")


class FinalAnswerException(Exception):
    def __init__(self, value):
        self.value = value


def evaluate_python_code(
    code: str,
    static_tools: Optional[Dict[str, Callable]] = None,
    custom_tools: Optional[Dict[str, Callable]] = None,
    state: Optional[Dict[str, Any]] = None,
    authorized_imports: List[str] = BASE_BUILTIN_MODULES,
    max_print_outputs_length: int = DEFAULT_MAX_LEN_OUTPUT,
):
    """
    Evaluate a python expression using the content of the variables stored in a state and only evaluating a given set
    of functions.

    This function will recurse through the nodes of the tree provided.

    Args:
        code (`str`):
            The code to evaluate.
        static_tools (`Dict[str, Callable]`):
            The functions that may be called during the evaluation. These can also be agents in a multiagent setting.
            These tools cannot be overwritten in the code: any assignment to their name will raise an error.
        custom_tools (`Dict[str, Callable]`):
            The functions that may be called during the evaluation.
            These tools can be overwritten in the code: any assignment to their name will overwrite them.
        state (`Dict[str, Any]`):
            A dictionary mapping variable names to values. The `state` should contain the initial inputs but will be
            updated by this function to contain all variables as they are evaluated.
            The print outputs will be stored in the state under the key "_print_outputs".
        authorized_imports (`List[str]`):
            The list of modules that can be imported by the code. By default, only a few safe modules are allowed.
            If it contains "*", it will authorize any import. Use this at your own risk!
    """
    try:
        expression = ast.parse(code)
    except SyntaxError as e:
        raise InterpreterError(
            f"Code parsing failed on line {e.lineno} due to: {type(e).__name__}\n"
            f"{e.text}"
            f"{' ' * (e.offset or 0)}^\n"
            f"Error: {str(e)}"
        )

    if state is None:
        state = {}
    static_tools = static_tools.copy() if static_tools is not None else {}
    custom_tools = custom_tools if custom_tools is not None else {}
    result = None
    state["_print_outputs"] = PrintContainer()
    state["_operations_count"] = {"counter": 0}

    if "final_answer" in static_tools:
        previous_final_answer = static_tools["final_answer"]

        def final_answer(answer):  # Using 'answer' as the argument like in the original function
            raise FinalAnswerException(previous_final_answer(answer))

        static_tools["final_answer"] = final_answer

    try:
        for node in expression.body:
            result = evaluate_ast(node, state, static_tools, custom_tools, authorized_imports)
        state["_print_outputs"].value = truncate_content(
            str(state["_print_outputs"]), max_length=max_print_outputs_length
        )
        is_final_answer = False
        return result, is_final_answer
    except FinalAnswerException as e:
        state["_print_outputs"].value = truncate_content(
            str(state["_print_outputs"]), max_length=max_print_outputs_length
        )
        is_final_answer = True
        return e.value, is_final_answer
    except Exception as e:
        state["_print_outputs"].value = truncate_content(
            str(state["_print_outputs"]), max_length=max_print_outputs_length
        )
        raise InterpreterError(
            f"Code execution failed at line '{ast.get_source_segment(code, node)}' due to: {type(e).__name__}: {e}"
        )


class PythonExecutor:
    pass


class LocalPythonExecutor(PythonExecutor):
    def __init__(
        self,
        additional_authorized_imports: List[str],
        max_print_outputs_length: Optional[int] = None,
    ):
        self.custom_tools = {}
        self.state = {}
        self.max_print_outputs_length = max_print_outputs_length
        if max_print_outputs_length is None:
            self.max_print_outputs_length = DEFAULT_MAX_LEN_OUTPUT
        self.additional_authorized_imports = additional_authorized_imports
        self.authorized_imports = list(set(BASE_BUILTIN_MODULES) | set(self.additional_authorized_imports))
        # TODO: assert self.authorized imports are all installed locally
        self.static_tools = None

    def __call__(self, code_action: str) -> Tuple[Any, str, bool]:
        output, is_final_answer = evaluate_python_code(
            code_action,
            static_tools=self.static_tools,
            custom_tools=self.custom_tools,
            state=self.state,
            authorized_imports=self.authorized_imports,
            max_print_outputs_length=self.max_print_outputs_length,
        )
        logs = str(self.state["_print_outputs"])
        return output, logs, is_final_answer

    def send_variables(self, variables: dict):
        self.state.update(variables)

    # PISEK COMMENTED IT OUT
    # def send_tools(self, tools: Dict[str, Tool]):
    #     self.static_tools = {**tools, **BASE_PYTHON_TOOLS.copy()}


def local_python_executor(code: str, authorized_imports: List[str]):
    """
    Executes Python code in a sandboxed environment with restricted imports for security.

    This function provides a simplified interface to the LocalPythonExecutor class, allowing
    for safe execution of Python code with controlled access to imports. It evaluates the
    provided code string and returns the result of the execution.

    Args:
        code (str):
            The Python code to execute as a string. This can be any valid Python code,
            including multiple statements, function definitions, class definitions, etc.
        authorized_imports (List[str]):
            A list of module names that are allowed to be imported by the code.
            These are in addition to the base built-in modules defined in BASE_BUILTIN_MODULES.
            For unrestricted imports (use with caution), include "*" in the list.

    Returns:
        Any: The result of the last statement in the executed code. If the code raises
             an exception, an InterpreterError will be raised with details about the error.

    Raises:
        InterpreterError: When the code execution fails due to syntax errors, unauthorized
                          imports, or other runtime errors.

    Examples:
        Basic arithmetic:
        >>> local_python_executor("2 + 2", [])
        4

        Using built-in modules:
        >>> local_python_executor("import math; math.sqrt(16)", [])
        4.0

        Using additional authorized imports:
        >>> local_python_executor("import numpy as np; np.array([1, 2, 3]).mean()", ["numpy"])
        2.0

        Function definition and execution:
        >>> code = '''
        ... def factorial(n):
        ...     return 1 if n <= 1 else n * factorial(n-1)
        ... factorial(5)
        ... '''
        >>> local_python_executor(code, [])
        120

        Working with data structures:
        >>> local_python_executor("data = {'a': 1, 'b': 2}; data['a'] + data['b']", [])
        3
    """
    tool = LocalPythonExecutor(additional_authorized_imports=authorized_imports)
    output, logs, is_final_answer = tool(code_action=code)
    return output


__all__ = ["evaluate_python_code", "LocalPythonExecutor"]


# Agent

In [ ]:
import os, sys
sys.path.insert(0, os.path.abspath("."))
from typing import Annotated, List
from typing_extensions import TypedDict
from logging import getLogger
logger = getLogger(__name__)

# LangGraph and LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
from langchain.tools import StructuredTool
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

# from langgraph_agents.tools.local_python_executor import local_python_executor, BASE_BUILTIN_MODULES

from dotenv import load_dotenv
from pathlib import Path

# current_file = Path(__file__).resolve()
# project_root = current_file.parent.parent
# try:
#     load_dotenv(project_root / "config/.env")
# except Exception as e:
#     print(f"Warning: Failed to load .env file: {e}")

try:
    OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
    if not OPENAI_API_KEY:
        raise ValueError("OPENAI_API_KEY not found in environment variables")
except Exception as e:
    logger.error(f"Error loading OPENAI_API_KEY: {e}")
    OPENAI_API_KEY = None

from pydantic import BaseModel, Field
from typing import Literal
from pydantic.types import SecretStr

# DEFAULT_MODEL_NAME = "gpt-4o-mini"
DEFAULT_MODEL_NAME = "gpt-4o"

DEFAULT_SYSTEM_PROMPT = """You are kintern, an advanced data analyst assistant specializing in data analysis, visualization, and problem-solving. Follow these guidelines for all tasks:

### Core Approach
Use the ReAct framework (Reasoning + Acting):
1. **Thought**: Break down the problem step-by-step. Identify required information and plan the solution.
2. **Action**: Select and execute the most appropriate tool or action based on your reasoning.
3. **Observation**: Evaluate the results of your action. Assess success, gather insights, and determine next steps.
4. **Repeat**: Iterate through Thought → Action → Observation until the problem is resolved.

**Example**:
User: "Analyze the relationship between two variables in a dataset."
- **Thought**: Examine data structure, then calculate correlation.
- **Action**: Use `python_tool` to load data and compute correlations.
- **Observation**: Correlation (r=0.85) between variables X and Y.
- **Thought**: Visualize the relationship for clarity.
- **Action**: Use `python_tool` to create a scatter plot with a regression line.

### General Guidelines
- Provide clear, concise explanations of your reasoning.
- Use appropriate tools for the task.
- Verify solutions when possible.
- Present results clearly and concisely.
- For code, enclose it in a code block with the language specified, e.g., ```python <code>```.
- Available libraries: {BASE_BUILTIN_MODULES}.
- Use `python_tool` to execute Python code for data tasks (loading, cleaning, analyzing, plotting, etc.) unless the user specifies otherwise.

### File Uploads & Data Analysis
When a user uploads a file, it is saved to a server path, and you will receive the absolute path with the user's question.

**Workflow**:
1. **Acknowledge the File Path**: Note the provided path.
2. **Load Data**: Use the exact file path in `python_tool` to load the data into a pandas DataFrame.
3. **Analyze**: Perform the requested analysis (e.g., calculations, aggregations).
4. **Respond**: Provide results based on the analysis.

**Example**:
User: "What are the total sales?" with file at a specific path.
- **Thought**: Load the CSV using the provided path, calculate total sales.
- **Action** (in `python_tool`):
```python
import pandas as pd
file_path = 'C:/path/to/file.csv'  # Use the exact path provided
df = pd.read_csv(file_path)
df['TotalSale'] = df['Quantity'] * df['Price']
total_sales = df['TotalSale'].sum()
total_sales
```

### 2. Sample Query for the California Housing Dataset

Now, let's create a sample query for your file. You can use this curl command:

```bash
curl -X POST "http://localhost:8000/api/v1/analysis/analyze" \
-H "Content-Type: application/json" \
-d '{
  "query": "Create a correlation heatmap of all features and describe the key insights from the data",
  "file_path": "california_housing_test.csv"
}'
```

Or if you're using Python:

```python
import requests
import json

url = "http://localhost:8000/api/v1/analysis/analyze"

payload = {
    "query": "Create a correlation heatmap of all features and describe the key insights from the data",
    "file_path": "california_housing_test.csv"
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, headers=headers, data=json.dumps(payload))
print(response.json())
```

### 3. Fix the Path Handling in the Agent

If you're still having issues after updating the system prompt, you might need to modify the `process_analysis_request` method in `analysis.py` to ensure it correctly handles Windows paths:

```python:c%3A%5CUsers%5Cuser%5CDesktop%5CTR%5Capp%5Cservices%5Canalysis.py
async def process_analysis_request(self,
                              query: str,
                              file_path: Optional[str] = None,
                              code: Optional[str] = None,
                              thread_id: str = "default") -> Dict[str, Any]:
    # Prepare the query with file path information if provided
    full_query = query
    if file_path:
        # Convert relative path to absolute path
        abs_file_path = os.path.join(UPLOAD_DIR, os.path.basename(file_path))
        if os.path.exists(abs_file_path):
            # Use forward slashes for compatibility with pandas
            normalized_path = abs_file_path.replace('\\', '/')
            full_query = f"Analyze the data from this file: {normalized_path}\n\n{query}"

    # Add custom code if provided
    if code:
        full_query = f"{full_query}\n\nUse this code as a starting point:\n```python\n{code}\n```"

    # Run the agent
    response = self.agent.analyze(full_query, thread_id)

    # Extract the result
    messages = response.get("messages", [])
    last_message = messages[-1] if messages else None
    content = last_message.content if last_message else ""

    # Check if there are any plots to convert to base64
    plt.close('all')  # Close any existing plots

    return {
        "result": content,
        "messages": [{
            "role": msg.type,
            "content": msg.content
        } for msg in messages]
    }```
"""

DEFAULT_AUTHORIZED_IMPORTS = ['sqlalchemy', 'pandas','os','sys','matplotlib.pyplot','seaborn']

# Define the state for our graph
class AgentState(TypedDict):
    messages: Annotated[List, add_messages]

def get_llm(api_key: str = OPENAI_API_KEY, model_name: str = "gpt-4o-mini", **kwargs):
    """Create an LLM instance"""
    return ChatOpenAI(
        api_key=api_key,
        model=model_name,
        **kwargs
    )

DEFAULT_LLM = get_llm()

def create_agent_builder(
    llm=DEFAULT_LLM,
    tools: List = [],
    system_prompt: str = DEFAULT_SYSTEM_PROMPT,
    authorized_imports: List[str] = DEFAULT_AUTHORIZED_IMPORTS
    ):

    authorized_imports = list(set(BASE_BUILTIN_MODULES) | set(authorized_imports))

    def _local_python_executor(code: str):
        """Execute Python code safely with restricted imports.

        Args:
            code (str): The code to execute.

        Returns:
            The result of the execution.
        """
        return local_python_executor(code, authorized_imports)

    python_tool = StructuredTool.from_function(
        func=_local_python_executor,
        name="python_tool",
        description="Execute Python code. Inputs: code (str)."
        )

    DEFAULT_TOOLS = [python_tool]

    tools += DEFAULT_TOOLS
    tools_node = ToolNode(tools=tools)
    llm = llm.bind_tools(tools=tools)

    # Define the nodes
    def llm_node(state: AgentState) -> AgentState:
        messages = state["messages"]
        payload = [SystemMessage(content=system_prompt)] + messages
        response = llm.invoke(payload)
        return {"messages": response}

    LLM_NODE = "LLM_NODE"
    TOOLS_NAME = "tools"

    builder = StateGraph(AgentState)
    builder.add_node(LLM_NODE, llm_node)
    builder.add_node(TOOLS_NAME, tools_node)

    builder.add_conditional_edges(
        LLM_NODE,
        tools_condition
    )
    builder.add_edge(TOOLS_NAME, LLM_NODE)
    builder.set_entry_point(LLM_NODE)

    return builder

if __name__ == "__main__":
    builder = create_agent_builder()

In [ ]:
import os, sys
sys.path.insert(0, " .. ")
# from src.agents.data_analyst import create_agent_builder, get_llm
# from src.utils.langgraph_helper import visualize_graph
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
llm = get_llm(model_name='gpt-4o', temperature=0.5)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x78317dea5e10>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x78317de63250>, root_client=<openai.OpenAI object at 0x78317de73b10>, root_async_client=<openai.AsyncOpenAI object at 0x78317dea5350>, model_name='gpt-4o', temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [ ]:
agent_builder = create_agent_builder(llm)
agent_builder

In [ ]:
agent = agent_builder.compile(MemorySaver() )
config = {"configurable": {"thread_id": "1"}}

In [ ]:
responses = agent.invoke({"messages": [{"role": "user", "content": "make a heatmap of features corrolatins chart from this file /content/temp/kintern_uploads/california_housing_test.csv"}]}, config=config)
responses ['messages' ]

/tmp/ipython-input-5-3787912382.py:1126: DeprecationWarning: Please import `ConstantWarning` from the `scipy.constants` namespace; the `scipy.constants.codata` namespace is deprecated and will be removed in SciPy 2.0.0.
  attr_value = getattr(raw_module, attr_name)
/tmp/ipython-input-5-3787912382.py:1126: DeprecationWarning: Please import `c` from the `scipy.constants` namespace; the `scipy.constants.codata` namespace is deprecated and will be removed in SciPy 2.0.0.
  attr_value = getattr(raw_module, attr_name)
/tmp/ipython-input-5-3787912382.py:1126: DeprecationWarning: Please import `find` from the `scipy.constants` namespace; the `scipy.constants.codata` namespace is deprecated and will be removed in SciPy 2.0.0.
  attr_value = getattr(raw_module, attr_name)
/tmp/ipython-input-5-3787912382.py:1126: DeprecationWarning: Please import `k` from the `scipy.constants` namespace; the `scipy.constants.codata` namespace is deprecated and will be removed in SciPy 2.0.0.
  attr_value = getatt

[HumanMessage(content='make a heatmap of features corrolatins chart from this file /content/temp/kintern_uploads/california_housing_test.csv', additional_kwargs={}, response_metadata={}, id='30e47fe8-1cf9-4710-9c40-be1488025de2'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Bm7R8pzfXpcEKRJsFXc1BHfG', 'function': {'arguments': '{"code":"import pandas as pd\\nimport seaborn as sns\\nimport matplotlib.pyplot as plt\\n\\n# Load the dataset\\ndf = pd.read_csv(\'/content/temp/kintern_uploads/california_housing_test.csv\')\\n\\n# Calculate the correlation matrix\\ncorrelation_matrix = df.corr()\\n\\n# Create a heatmap\\nplt.figure(figsize=(10, 8))\\nsns.heatmap(correlation_matrix, annot=True, cmap=\'coolwarm\', fmt=\'.2f\')\\nplt.title(\'Correlation Heatmap of California Housing Features\')\\nplt.show()"}', 'name': 'python_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 132, 'prompt_tokens': 1264, 'total_tokens'

# GradioUI

In [ ]:
import gradio as gr
from langchain_core.messages import AIMessage
from typing import Any, Optional, Dict
import os
import shutil
import re

class GradioUI:
    """
    UI class that correctly handles streaming for both text-only and
    multimodal (text + chart) agent responses.
    """
    def __init__(self, agent: Any, config: dict):
        self.agent = agent
        self.config = config
        self.upload_dir = "/content/temp/kintern_uploads"
        os.makedirs(self.upload_dir, exist_ok=True)
        print(f"File uploads and charts will be saved to: {self.upload_dir}")

    def _chat_handler(self, message: Dict[str, Any], history: list):
        """
        Processes user input and yields either a string (for text) or a
        tuple (for text + chart) to avoid Gradio history errors.
        """
        user_text = message['text']
        uploaded_files = message['files']
        final_message_for_agent = user_text

        if uploaded_files:
            temp_path = uploaded_files[0]
            original_filename = os.path.basename(temp_path)
            permanent_path = os.path.join(self.upload_dir, original_filename)
            shutil.copy(temp_path, permanent_path)
            print(f"File '{original_filename}' has been saved to '{permanent_path}'.")

            file_context = f"""The user has uploaded a file named '{original_filename}'.
It is available for you to access at the following absolute path:
---
{permanent_path}
---
"""
            final_message_for_agent = f"{file_context}\n\nUser Question: {user_text}"

        print(f"Final message to agent:\n---\n{final_message_for_agent}\n---")

        inputs = {"messages": [("user", final_message_for_agent)]}
        full_response = ""

        for chunk in self.agent.stream(inputs, self.config):
            for node_name, state_update in chunk.items():
                if "messages" in state_update:
                    messages_in_update = state_update["messages"]
                    last_message = messages_in_update[-1] if isinstance(messages_in_update, list) else messages_in_update

                    if isinstance(last_message, AIMessage) and not last_message.tool_calls:
                        full_response = last_message.content

                        # === THE FIX IS HERE ===
                        # Check for the chart string, but decide what to yield based on the result.

                        match = re.search(r"CHART:(.*)", full_response)
                        if match:
                            # A chart is present! Yield a TUPLE.
                            chart_path = match.group(1).strip()
                            text_for_ui = full_response.replace(match.group(0), "").strip()
                            yield (text_for_ui, chart_path)
                        else:
                            # NO chart is present. Yield a simple STRING.
                            yield full_response

    def launch(self):
        """Builds and launches the Gradio chat interface."""
        ui = gr.ChatInterface(
            fn=self._chat_handler,
            title="kintern - Data Analyst Assistant",
            description="Upload a CSV file and ask me to analyze it or create a chart from it.",
            multimodal=True,
        )

        print("Launching Gradio UI...")
        ui.launch(share=True, debug=True)

In [ ]:
agent = agent_builder.compile(MemorySaver())
config = {"configurable": {"thread_id": "1"}}

app = GradioUI(agent, config)
app.launch()

File uploads and charts will be saved to: /content/temp/kintern_uploads


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Launching Gradio UI...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://cdcfb9bfb3581ec0cf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Final message to agent:
---
what is the log 5120 in base 10
---
File 'california_housing_test.csv' has been saved to '/content/temp/kintern_uploads/california_housing_test.csv'.
Final message to agent:
---
The user has uploaded a file named 'california_housing_test.csv'.
It is available for you to access at the following absolute path:
---
/content/temp/kintern_uploads/california_housing_test.csv
---


User Question: gimme a basic analysis of  this data 
---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           3000 non-null   float64
 1   latitude            3000 non-null   float64
 2   housing_median_age  3000 non-null   float64
 3   total_rooms         3000 non-null   float64
 4   total_bedrooms      3000 non-null   float64
 5   population          3000 non-null   float64
 6   households          3000 non-null 

# new and need to debug


## agent

In [ ]:
import os, sys
sys.path.insert(0, os.path.abspath("."))
from typing import Annotated, List, Optional, Dict, Any
from typing_extensions import TypedDict
from logging import getLogger
import traceback

logger = getLogger(__name__)

# LangGraph and LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from langchain.tools import StructuredTool
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

# from langgraph_agents.tools.local_python_executor import local_python_executor, BASE_BUILTIN_MODULES

from dotenv import load_dotenv
from pathlib import Path
from pydantic import BaseModel, Field
from typing import Literal
from pydantic.types import SecretStr

# Load environment variables
# current_file = Path(__file__).resolve()
# project_root = current_file.parent.parent

def load_api_key() -> Optional[str]:
    """Safely load OpenAI API key with proper error handling"""
    try:
        load_dotenv()
        api_key = os.environ.get('OPENAI_API_KEY')
        if not api_key:
            logger.warning("OPENAI_API_KEY not found in environment variables")
            return None
        logger.info("Successfully loaded OPENAI_API_KEY")
        return api_key
    except Exception as e:
        logger.error(f"Error loading environment variables: {e}")
        return None

OPENAI_API_KEY = load_api_key()

# Configuration
DEFAULT_MODEL_NAME = "gpt-4o"
DEFAULT_SYSTEM_PROMPT = "You are a helpful AI assistant with Python execution capabilities."
DEFAULT_AUTHORIZED_IMPORTS = ['sqlalchemy', 'dotenv', 'os', 'sys', 'pandas']

class Valves(BaseModel):
    """Valves used for Open WebUI's Pipeline"""
    MODEL_NAME: Literal["gpt-4o-mini", "gpt-4o", "o3-mini"] = Field(default=DEFAULT_MODEL_NAME, description="OpenAI")
    AUTHORIZED_IMPORTS: str = Field(default=", ".join(DEFAULT_AUTHORIZED_IMPORTS), description="Authorized imports")

# Enhanced state with error handling capabilities
class AgentState(TypedDict):
    messages: Annotated[List, add_messages]
    error_count: int
    last_error: Optional[str]
    is_recovering: bool
    max_retries: int

class AgentError(Exception):
    """Custom exception for agent-specific errors"""
    def __init__(self, message: str, error_type: str = "general", recoverable: bool = True):
        super().__init__(message)
        self.error_type = error_type
        self.recoverable = recoverable

def get_llm(api_key: str = OPENAI_API_KEY, model_name: str = "gpt-4o-mini", **kwargs):
    """Create an LLM instance with validation"""
    if not api_key:
        raise AgentError("OpenAI API key is required", "configuration", False)

    try:
        return ChatOpenAI(
            api_key=api_key,
            model=model_name,
            **kwargs
        )
    except Exception as e:
        logger.error(f"Failed to initialize LLM: {e}")
        raise AgentError(f"LLM initialization failed: {str(e)}", "llm_init", False)

def create_agent_builder(
    llm=None,
    tools: List = None,
    system_prompt: str = DEFAULT_SYSTEM_PROMPT,
    authorized_imports: List[str] = None,
    max_retries: int = 3
):
    """Create an agent builder with comprehensive error handling"""

    # Initialize defaults with proper null checks
    if tools is None:
        tools = []
    if authorized_imports is None:
        authorized_imports = DEFAULT_AUTHORIZED_IMPORTS.copy()
    if llm is None:
        llm = get_llm()

    # Merge authorized imports with base modules
    authorized_imports = list(set(BASE_BUILTIN_MODULES) | set(authorized_imports))

    def _safe_python_executor(code: str) -> str:
        """Execute Python code with comprehensive error handling"""
        try:
            result = local_python_executor(code, authorized_imports)
            return str(result) if result is not None else "Code executed successfully (no output)"
        except ImportError as e:
            error_msg = f"Import error: {str(e)}. Allowed imports: {', '.join(authorized_imports)}"
            logger.warning(error_msg)
            raise AgentError(error_msg, "import_error", True)
        except SyntaxError as e:
            error_msg = f"Syntax error in code: {str(e)}"
            logger.warning(error_msg)
            raise AgentError(error_msg, "syntax_error", True)
        except Exception as e:
            error_msg = f"Code execution error: {str(e)}"
            logger.error(f"{error_msg}\nTraceback: {traceback.format_exc()}")
            raise AgentError(error_msg, "execution_error", True)

    # Create Python tool
    python_tool = StructuredTool.from_function(
        func=_safe_python_executor,
        name="python_tool",
        description="Execute Python code safely with restricted imports. Input: code (str)."
    )

    # Combine tools
    all_tools = [python_tool] + tools
    tools_node = ToolNode(tools=all_tools)
    llm_with_tools = llm.bind_tools(tools=all_tools)

    def llm_node(state: AgentState) -> AgentState:
        """LLM node with error handling and recovery logic"""
        try:
            messages = state["messages"]
            error_count = state.get("error_count", 0)
            last_error = state.get("last_error")
            is_recovering = state.get("is_recovering", False)

            # Prepare system message with error context if recovering
            system_content = system_prompt
            if is_recovering and last_error:
                system_content += f"\n\nNote: The previous operation failed with error: {last_error}. Please adjust your approach accordingly."

            payload = [SystemMessage(content=system_content)] + messages
            response = llm_with_tools.invoke(payload)

            # Reset error state on successful LLM call
            return {
                "messages": [response],
                "error_count": 0,
                "last_error": None,
                "is_recovering": False
            }

        except Exception as e:
            error_msg = f"LLM node error: {str(e)}"
            logger.error(error_msg)

            error_response = AIMessage(
                content=f"I encountered an error: {error_msg}. Let me try a different approach."
            )

            return {
                "messages": [error_response],
                "error_count": error_count + 1,
                "last_error": error_msg,
                "is_recovering": True
            }

    def tools_node_with_error_handling(state: AgentState) -> AgentState:
        """Tools node wrapper with error handling"""
        try:
            result = tools_node.invoke(state)
            # Reset error state on successful tool execution
            updated_result = dict(result)
            updated_result.update({
                "error_count": 0,
                "last_error": None,
                "is_recovering": False
            })
            return updated_result

        except AgentError as e:
            error_count = state.get("error_count", 0) + 1
            logger.warning(f"Tool execution failed (attempt {error_count}): {e}")

            # Create error message for the conversation
            error_message = ToolMessage(
                content=f"Tool execution failed: {str(e)}",
                tool_call_id="error_" + str(error_count)
            )

            return {
                "messages": [error_message],
                "error_count": error_count,
                "last_error": str(e),
                "is_recovering": True
            }
        except Exception as e:
            error_count = state.get("error_count", 0) + 1
            error_msg = f"Unexpected tool error: {str(e)}"
            logger.error(f"{error_msg}\nTraceback: {traceback.format_exc()}")

            error_message = ToolMessage(
                content=f"Unexpected error occurred: {error_msg}",
                tool_call_id="error_" + str(error_count)
            )

            return {
                "messages": [error_message],
                "error_count": error_count,
                "last_error": error_msg,
                "is_recovering": True
            }

    def should_continue(state: AgentState) -> str:
        """Enhanced routing logic with error handling"""
        messages = state["messages"]
        error_count = state.get("error_count", 0)
        state_max_retries = state.get("max_retries", max_retries)  # Use the parameter passed to the function

        # Check if we've exceeded max retries
        if error_count >= state_max_retries:
            logger.error(f"Max retries ({state_max_retries}) exceeded, ending conversation")
            return END

        # Standard tools condition logic
        last_message = messages[-1]
        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            return "tools"
        return END

    # Node names
    LLM_NODE = "llm_node"
    TOOLS_NODE = "tools_node"

    # Build the graph
    builder = StateGraph(AgentState)
    builder.add_node(LLM_NODE, llm_node)
    builder.add_node(TOOLS_NODE, tools_node_with_error_handling)

    # Add edges with enhanced routing
    builder.add_conditional_edges(
        LLM_NODE,
        should_continue,
        {
            "tools": TOOLS_NODE,
            END: END
        }
    )
    builder.add_edge(TOOLS_NODE, LLM_NODE)
    builder.set_entry_point(LLM_NODE)

    return builder

def create_agent_with_checkpointing(
    llm=None,
    tools: List = None,
    system_prompt: str = DEFAULT_SYSTEM_PROMPT,
    authorized_imports: List[str] = None,
    max_retries: int = 3
):
    """Create an agent with memory checkpointing for persistence"""
    builder = create_agent_builder(
        llm=llm,
        tools=tools,
        system_prompt=system_prompt,
        authorized_imports=authorized_imports,
        max_retries=max_retries
    )

    # Add memory saver for checkpointing
    memory = MemorySaver()
    agent = builder.compile(checkpointer=memory)

    return agent

def run_agent_safely(agent, messages: List, thread_id: str = "default"):
    """Safely run the agent with proper error handling"""
    try:
        initial_state = {
            "messages": messages,
            "error_count": 0,
            "last_error": None,
            "is_recovering": False,
            "max_retries": 3
        }

        config = {"configurable": {"thread_id": thread_id}}

        result = agent.invoke(initial_state, config=config)
        return result

    except Exception as e:
        logger.error(f"Agent execution failed: {e}")
        logger.error(f"Traceback: {traceback.format_exc()}")
        return {
            "messages": [AIMessage(content=f"I encountered a critical error: {str(e)}")],
            "error_count": float('inf'),
            "last_error": str(e),
            "is_recovering": False
        }

## instance

In [ ]:
agent = create_agent_with_checkpointing()

## Gradio UI

In [ ]:
import gradio as gr
from langchain_core.messages import AIMessage, HumanMessage
from typing import Any, Optional, Dict
import os
import shutil
import re
import uuid
from logging import getLogger

# Import your improved agent functions
# from your_agent_module import create_agent_with_checkpointing, run_agent_safely, AgentError

logger = getLogger(__name__)

class GradioUI:
    """
    Enhanced UI class that handles streaming with robust error handling
    for both text-only and multimodal (text + chart) agent responses.
    """
    def __init__(self, agent: Any = None, config: dict = None, max_retries: int = 3):
        # Set max_retries first to ensure it's always available
        self.max_retries = max_retries

        # Create agent if not provided
        if agent is None:
            try:
                self.agent = create_agent_with_checkpointing(max_retries=max_retries)
                logger.info("Successfully created LangGraph agent")
            except Exception as e:
                logger.error(f"Failed to create agent: {e}")
                raise RuntimeError(f"Agent initialization failed: {e}")
        else:
            self.agent = agent

        # Set up configuration with thread management
        self.config = config or {"configurable": {"thread_id": "gradio_session"}}
        self.upload_dir = "/content/temp/kintern_uploads"

        # Create upload directory
        os.makedirs(self.upload_dir, exist_ok=True)
        print(f"File uploads and charts will be saved to: {self.upload_dir}")
        print(f"Max retries set to: {self.max_retries}")

    def _generate_thread_id(self) -> str:
        """Generate unique thread ID for each conversation"""
        return f"gradio_{uuid.uuid4().hex[:8]}"

    def _handle_file_upload(self, uploaded_files: list) -> Optional[str]:
        """Handle file upload with error handling"""
        if not uploaded_files:
            return None

        try:
            temp_path = uploaded_files[0]
            original_filename = os.path.basename(temp_path)
            permanent_path = os.path.join(self.upload_dir, original_filename)

            # Copy file safely
            shutil.copy(temp_path, permanent_path)
            logger.info(f"File '{original_filename}' saved to '{permanent_path}'")

            return f"""The user has uploaded a file named '{original_filename}'.
It is available for you to access at the following absolute path:
---
{permanent_path}
---
"""
        except Exception as e:
            logger.error(f"File upload failed: {e}")
            return f"Error uploading file: {str(e)}"

    def _extract_chart_info(self, response_text: str) -> tuple[str, Optional[str]]:
        """Extract chart information from response text"""
        match = re.search(r"CHART:(.*)", response_text)
        if match:
            chart_path = match.group(1).strip()
            text_for_ui = response_text.replace(match.group(0), "").strip()
            return text_for_ui, chart_path
        return response_text, None

    def _chat_handler(self, message: Dict[str, Any], history: list):
        """
        Enhanced chat handler with comprehensive error handling and recovery
        """
        try:
            user_text = message.get('text', '')
            uploaded_files = message.get('files', [])

            if not user_text and not uploaded_files:
                yield "Please provide a message or upload a file."
                return

            # Handle file upload
            file_context = self._handle_file_upload(uploaded_files)

            # Construct final message
            if file_context:
                final_message_for_agent = f"{file_context}\n\nUser Question: {user_text}"
            else:
                final_message_for_agent = user_text

            logger.info(f"Processing message: {final_message_for_agent[:100]}...")

            # Generate unique thread ID for this conversation
            thread_id = self._generate_thread_id()
            config_with_thread = {
                "configurable": {
                    "thread_id": thread_id
                }
            }

            # Prepare messages for the agent
            messages = [HumanMessage(content=final_message_for_agent)]

            # Get max_retries from instance variable
            max_retries_value = self.max_retries

            # Use the safe runner wrapper
            try:
                # Stream the agent response
                inputs = {
                    "messages": messages,
                    "error_count": 0,
                    "last_error": None,
                    "is_recovering": False,
                    "max_retries": max_retries_value
                }

                full_response = ""
                error_occurred = False

                for chunk in self.agent.stream(inputs, config_with_thread):
                    for node_name, state_update in chunk.items():
                        logger.debug(f"Processing chunk from {node_name}")

                        if "messages" in state_update:
                            messages_in_update = state_update["messages"]
                            last_message = messages_in_update[-1] if isinstance(messages_in_update, list) else messages_in_update

                            # Check for errors in state
                            error_count = state_update.get("error_count", 0)
                            last_error = state_update.get("last_error")

                            if error_count >= max_retries_value:
                                error_occurred = True
                                yield f"I encountered multiple errors and need to stop. Last error: {last_error}"
                                return

                            if isinstance(last_message, AIMessage) and not last_message.tool_calls:
                                full_response = last_message.content

                                # Handle chart detection and yield appropriate format
                                text_content, chart_path = self._extract_chart_info(full_response)

                                if chart_path:
                                    # Verify chart file exists before yielding
                                    if os.path.exists(chart_path):
                                        yield (text_content, chart_path)
                                    else:
                                        yield f"{text_content}\n\n(Chart generation attempted but file not found: {chart_path})"
                                else:
                                    yield full_response

                # If we didn't get any response, provide fallback
                if not full_response and not error_occurred:
                    yield "I'm processing your request but haven't generated a response yet. Please try again."

            except Exception as stream_error:
                logger.error(f"Streaming error: {stream_error}")

                # Fallback to non-streaming execution
                try:
                    logger.info("Attempting fallback to non-streaming execution")
                    result = run_agent_safely(self.agent, messages, thread_id)

                    if result and "messages" in result and result["messages"]:
                        last_msg = result["messages"][-1]
                        if hasattr(last_msg, 'content'):
                            text_content, chart_path = self._extract_chart_info(last_msg.content)
                            if chart_path and os.path.exists(chart_path):
                                yield (text_content, chart_path)
                            else:
                                yield text_content
                        else:
                            yield "I processed your request but couldn't generate a proper response."
                    else:
                        yield "I encountered an error processing your request. Please try again with a simpler query."

                except Exception as fallback_error:
                    logger.error(f"Fallback execution also failed: {fallback_error}")
                    yield f"I'm experiencing technical difficulties. Error: {str(fallback_error)}"

        except Exception as e:
            logger.error(f"Critical error in chat handler: {e}")
            yield f"A critical error occurred: {str(e)}. Please refresh and try again."

    def _create_error_recovery_handler(self):
        """Create a simple error recovery interface"""
        def reset_agent():
            try:
                self.agent = create_agent_with_checkpointing(max_retries=self.max_retries)
                return "Agent reset successfully. You can start a new conversation."
            except Exception as e:
                return f"Failed to reset agent: {str(e)}"

        return reset_agent

    def launch(self, share: bool = True, debug: bool = True, server_name: str = "0.0.0.0"):
        """Enhanced launch method with error recovery options - compatible with older Gradio versions"""

        # Create the main chat interface with basic parameters only
        try:
            chat_interface = gr.ChatInterface(
                fn=self._chat_handler,
                title="🤖 kintern - Data Analyst Assistant (Enhanced)",
                description="""
                Upload a CSV file and ask me to analyze it or create charts from it.

                **Features:**
                - Robust error handling and recovery
                - File upload support
                - Chart generation
                - Conversation persistence

                If you encounter issues, try rephrasing your question or use simpler requests.
                """,
                multimodal=True
            )
        except Exception as e:
            logger.error(f"Failed to create ChatInterface: {e}")
            # Fallback to basic ChatInterface
            chat_interface = gr.ChatInterface(
                fn=self._chat_handler,
                multimodal=True
            )

        # Create enhanced interface with custom controls
        with gr.Blocks(title="kintern - Data Analyst Assistant") as full_interface:
            gr.Markdown("# 🤖 kintern - Data Analyst Assistant (Enhanced)")
            gr.Markdown("""
            Upload a CSV file and ask me to analyze it or create charts from it.

            **Features:** Robust error handling, file upload support, chart generation, conversation persistence
            """)

            # Render the chat interface
            chat_interface.render()

            # Add custom controls section
            with gr.Row():
                with gr.Column(scale=1):
                    reset_btn = gr.Button("🔄 Reset Agent", variant="secondary", size="sm")
                with gr.Column(scale=3):
                    reset_output = gr.Textbox(
                        label="Status",
                        interactive=False,
                        show_label=False,
                        placeholder="Agent status will appear here..."
                    )

            # Wire up the reset functionality
            reset_btn.click(
                fn=self._create_error_recovery_handler(),
                outputs=reset_output
            )

            # Add troubleshooting section
            with gr.Accordion("🔧 Troubleshooting & Tips", open=False):
                gr.Markdown("""
                ### Common Issues & Solutions:
                - **Agent not responding?** → Click "Reset Agent" button above
                - **File upload errors?** → Ensure your file is a valid CSV format
                - **Complex requests failing?** → Break them into smaller, simpler parts
                - **Charts not showing?** → Check that file paths are accessible

                ### Usage Tips:
                - Start with simple requests to test functionality
                - Upload files before asking analysis questions
                - Be specific about what type of chart or analysis you want
                - If you see errors, try rephrasing your request
                """)

        print("Launching Enhanced Gradio UI with error handling...")
        print(f"Upload directory: {self.upload_dir}")
        print(f"Max retries per conversation: {self.max_retries}")

        try:
            full_interface.launch(
                share=share,
                debug=debug,
                server_name=server_name,
                show_error=True
            )
        except Exception as e:
            logger.error(f"Failed to launch Gradio interface: {e}")
            print(f"Launch failed: {e}")
            # Try basic launch as fallback
            try:
                print("Attempting basic launch as fallback...")
                chat_interface.launch(share=share, debug=debug)
            except Exception as fallback_error:
                logger.error(f"Fallback launch also failed: {fallback_error}")
                raise fallback_error

# Usage example and factory function
def create_gradio_app(
    system_prompt: str = None,
    authorized_imports: list = None,
    max_retries: int = 3,
    **gradio_kwargs
) -> GradioUI:
    """
    Factory function to create a configured Gradio app with the enhanced agent
    """
    try:
        # Create agent with custom configuration
        agent = create_agent_with_checkpointing(
            system_prompt=system_prompt,
            authorized_imports=authorized_imports,
            max_retries=max_retries
        )

        # Create and return Gradio UI
        ui = GradioUI(agent=agent, max_retries=max_retries)
        return ui

    except Exception as e:
        logger.error(f"Failed to create Gradio app: {e}")
        raise RuntimeError(f"App creation failed: {e}")


## RUN

In [ ]:
try:
  # Custom system prompt for data analysis
  custom_prompt =  DEFAULT_POSTGRES_PROMPT + FILE_UPLOAD_PROMPT_WITH_PATH + plotting_chart_prompt

  # Additional imports for data analysis
  # custom_imports = ['pandas', 'matplotlib', 'seaborn', 'numpy', 'plotly']

  # Create the app
  app = create_gradio_app(
      system_prompt=custom_prompt,
      max_retries=3
  )

  # Launch with custom settings
  app.launch(share=True, debug=True)

  # Launch with custom settings
  app.launch(share=True, debug=True)

except Exception as e:
  print(f"Failed to start application: {e}")
  logger.error(f"Application startup failed: {e}")

File uploads and charts will be saved to: /content/temp/kintern_uploads
Max retries set to: 3


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Launching Enhanced Gradio UI with error handling...
Upload directory: /content/temp/kintern_uploads
Max retries per conversation: 3
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://dd289c5c0c00571dc3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:__main__:Code execution error: Code execution failed at line 'print(f'CHART:{chart_path}')' due to: InterpreterError: It is not permitted to evaluate other functions than the provided tools or functions defined/imported in previous code (tried to execute print).
Traceback: Traceback (most recent call last):
  File "/tmp/ipython-input-95-1487955346.py", line 1482, in evaluate_python_code
    result = evaluate_ast(node, state, static_tools, custom_tools, authorized_imports)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-95-1487955346.py", line 295, in _check_return
    result = func(expression, state, static_tools, custom_tools, authorized_imports=authorized_imports)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-95-1487955346.py", line 1348, in evaluate_ast
    return evaluate_ast(expression.value, *common_params)
           ^^^^^^^

Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://dd289c5c0c00571dc3.gradio.live


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Launching Enhanced Gradio UI with error handling...
Upload directory: /content/temp/kintern_uploads
Max retries per conversation: 3
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://967d6d5e64332f0a40.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://967d6d5e64332f0a40.gradio.live


# orchestrator

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langgraph.graph import StateGraph, END
from typing import Annotated, List
from typing_extensions import TypedDict

# تعریف حالت برای ایجنت نویسنده
class WriterState(TypedDict):
    messages: Annotated[List, add_messages]

# پرامپت نویسنده
WRITER_SYSTEM_PROMPT = """
شما یک نویسنده حرفه‌ای هستید. وظیفه شما تولید یک گزارش متنی با کیفیت و ساختار استاندارد بر اساس ورودی است. گزارش باید شامل بخش‌های زیر باشد:
1. **مقدمه**: توضیح مختصر درباره موضوع گزارش.
2. **تحلیل**: شرح نتایج تحلیل (اگر ارائه شده باشد).
3. **نتیجه‌گیری**: جمع‌بندی و پیشنهادات.
ورودی: {input}
لطفاً گزارش را با زبان حرفه‌ای و روان بنویسید.
"""

def create_writer_builder(
    llm=ChatOpenAI(model="gpt-4o", api_key=os.environ['OPENAI_API_KEY']),
    system_prompt: str = WRITER_SYSTEM_PROMPT
):
    # تعریف نود نویسنده
    def writer_node(state: WriterState) -> WriterState:
        messages = state["messages"]
        payload = [SystemMessage(content=system_prompt)] + messages
        response = llm.invoke(payload)
        return {"messages": response}

    # تعریف گراف
    builder = StateGraph(WriterState)
    builder.add_node("writer", writer_node)
    builder.set_entry_point("writer")
    builder.add_edge("writer", END)
    return builder

# کامپایل گراف نویسنده
writer_builder = create_writer_builder()
writer_workflow = writer_builder.compile()

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
import json
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict
from typing import Annotated, List


class MainState(TypedDict):
    messages: Annotated[List, add_messages]
    next_agent: str


ORCHESTRATOR_PROMPT = """
شما یک هماهنگ‌کننده هستید. وظیفه شما بررسی درخواست کاربر و تصمیم‌گیری برای ارجاع به ایجنت مناسب است:
- اگر درخواست فقط شامل تحلیل داده، اجرای کد یا محاسبات باشد، به 'analyst' ارجاع دهید.
- اگر درخواست فقط شامل نوشتن متن، گزارش یا محتوای متنی باشد، به 'writer' ارجاع دهید.
درخواست کاربر: {input}
پاسخ دهید با فرمت: {{ "agent": "writer" }} یا {{ "agent": "analyst" }}
"""


orchestrator_llm = ChatOpenAI(model="gpt-4o", api_key=os.environ['OPENAI_API_KEY'])
orchestrator_prompt = ChatPromptTemplate.from_template(ORCHESTRATOR_PROMPT)
orchestrator = orchestrator_prompt | orchestrator_llm

def orchestrator_node(state: MainState) -> MainState:
    input_message = state["messages"][-1].content if state["messages"] else ""
    response = orchestrator.invoke({"input": input_message})
    decision = json.loads(response.content)
    state["next_agent"] = decision["agent"]
    state["messages"].append(SystemMessage(content=f"Orchestrator تصمیم گرفت به {decision['agent']} ارجاع دهد"))
    return state


def call_writer(state: MainState) -> MainState:
    input_message = state["messages"][-1].content if state["messages"] else ""
    writer_result = writer_workflow.invoke({"messages": [HumanMessage(content=input_message)]})
    state["messages"].extend(writer_result["messages"])
    return state

def call_analyst(state: MainState) -> MainState:
    input_message = state["messages"][-1].content if state["messages"] else ""
    analyst_result = create_agent_builder().compile().invoke({"messages": [HumanMessage(content=input_message)]})
    state["messages"].extend(analyst_result["messages"])
    return state

main_graph = StateGraph(MainState)
main_graph.add_node("orchestrator", orchestrator_node)
main_graph.add_node("writer", call_writer)
main_graph.add_node("analyst", call_analyst)
main_graph.set_entry_point("orchestrator")
main_graph.add_conditional_edges(
    "orchestrator",
    lambda state: state["next_agent"],
    {
        "writer": "writer",
        "analyst": "analyst"
    }
)
main_graph.add_edge("writer", END)
main_graph.add_edge("analyst", END)

main_workflow = main_graph.compile()

In [ ]:
input_text = "داده‌های زیر را تحلیل کنید و میانگین آن‌ها را محاسبه کنید: [10, 20, 30, 40, 50] سپس یک گزارش از آن ایجاد کنید"
result = main_workflow.invoke({"messages": [HumanMessage(content=input_text)], "next_agent": ""})
print("\nنتیجه برای تحلیلگر داده:")
for message in result["messages"]:
    print(message.content)


نتیجه برای تحلیلگر داده:
داده‌های زیر را تحلیل کنید و میانگین آن‌ها را محاسبه کنید: [10, 20, 30, 40, 50] سپس یک گزارش از آن ایجاد کنید
Orchestrator تصمیم گرفت به analyst ارجاع دهد
Orchestrator تصمیم گرفت به analyst ارجاع دهد
به نظر می‌رسد که شما می‌خواهید اطلاعات بیشتری درباره تحلیلی خاص یا داده‌هایی که باید تحلیل شوند، ارائه دهید. لطفاً بفرمایید که چه نوع تحلیلی مد نظر دارید یا آیا داده‌هایی را برای تحلیل دارید؟ اگر داده‌هایی دارید، لطفاً آن‌ها را بارگذاری کنید و من در خدمت شما خواهم بود.
